### Import relevant functions

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
import itertools
from collections import Counter
from collections import defaultdict

### List sets of primers and probes for variant detection

In [2]:
#PCR primers
#Total_dict = {'For': 'GACCCCAAAATCAGCGAAAT',
#               'Rev':'CAGATTCAACTGGCAGTAACCAGA',
#              'probe':'ACCCCGCATTACGTTTGGTGGACC'}  
Total_dict = {'For': 'TTACAAACATTGGCCGCAAA',
               'Rev':'TTCTTCGGAATGTCGCGC',
              'probe':'ACAATTTGCCCCCAGCGCTTCAG'}  
Omicron_dict = {'For':'CTTGTTAAACAACTTAGCTCCAAA',
               'Rev':'GATAGGTTGATCACAGGCAG',
              'probe':'CGTCTTGACAAAGTTGAGGCTGAAGTGCA'}  

### Processing FASTA file format

In [3]:
FASTA_file_1 = 'raw_FASTA/Vietnam.fasta'


FASTA_dict = {}
entry = False

with open(FASTA_file_1,'r') as f:
    for line in f:
        if line.startswith('>'):
            if entry:
                FASTA_dict[seq_name] = seq
            seq_name = line.lstrip('>').rstrip('\n')
            seq = ''
            entry = True
        else:
            seq += line.rstrip('\n')

 
            
len(FASTA_dict)

2007

### Checking if primers works for each virus sequence from GISAID data

In [4]:
#if each of primers/probe sets work, the function will return True. 


Total_seq_list = list(Total_dict.values())
Omicron_seq_list = list(Omicron_dict.values())


def do_primers_work():
    variant_qPCR_results = {}
    for seq_key in FASTA_dict.keys():
        variant_qPCR_results[seq_key] = []
        number_Total = 0
        number_Omicron = 0

        for target_seq1 in Total_seq_list:
            if FASTA_dict[seq_key].find(target_seq1) != -1:
                number_Total += 1
        if number_Total == 3:
            variant_qPCR_results[seq_key].append(True)
        else:
            variant_qPCR_results[seq_key].append(False)                    
 
        for target_seq2 in Omicron_seq_list:
            if FASTA_dict[seq_key].find(target_seq2) != -1:
                number_Omicron += 1
        if number_Omicron == 3:
            variant_qPCR_results[seq_key].append(True)
        else:
            variant_qPCR_results[seq_key].append(False)            


    
    return(variant_qPCR_results)

do_primers_work()

{'hCoV-19/Vietnam/HCMC-05049-02/2021': [True, False],
 'hCoV-19/Vietnam/39730/2020': [True, False],
 'hCoV-19/Vietnam/HCMC-05047-02/2021': [True, False],
 'hCoV-19/Vietnam/HCMC-05109070-05/2021': [True, False],
 'hCoV-19/Vietnam/040112/2021': [True, False],
 'hCoV-19/Vietnam/050112/2021': [True, False],
 'hCoV-19/Vietnam/060112/2021': [True, False],
 'hCoV-19/Vietnam/080112/2021': [True, False],
 'hCoV-19/Vietnam/090112/2021': [True, False],
 'hCoV-19/Vietnam/110112/2021': [True, False],
 'hCoV-19/Vietnam/140112/2021': [True, False],
 'hCoV-19/Vietnam/030113/2021': [True, False],
 'hCoV-19/Vietnam/070113/2021': [True, False],
 'hCoV-19/Vietnam/080113/2021': [True, False],
 'hCoV-19/Vietnam/110113/2021': [True, False],
 'hCoV-19/Vietnam/160113/2021': [True, False],
 'hCoV-19/Vietnam/170113/2021': [True, False],
 'hCoV-19/Vietnam/190113/2021': [True, False],
 'hCoV-19/Vietnam/200113/2021': [True, False],
 'hCoV-19/Vietnam/010114/2021': [True, False],
 'hCoV-19/Vietnam/050114/2021': [True

### Classification of virus sequences based on decision tree

In [5]:
#Based on the results above, we can categorize each virus sequence into each type of variants.
def decision_tree():
    variant_classification = {}
    variant_qPCR_results = do_primers_work()
    for seq_key in variant_qPCR_results.keys():
        if variant_qPCR_results[seq_key] == [True, True]:
            variant_classification[seq_key] = 'Omicron'
        elif variant_qPCR_results[seq_key] == [False, True]:
            variant_classification[seq_key] = 'Omicron'
        else:    
            variant_classification[seq_key] = 'Others'            
    return(variant_classification)

decision_tree()

{'hCoV-19/Vietnam/HCMC-05049-02/2021': 'Others',
 'hCoV-19/Vietnam/39730/2020': 'Others',
 'hCoV-19/Vietnam/HCMC-05047-02/2021': 'Others',
 'hCoV-19/Vietnam/HCMC-05109070-05/2021': 'Others',
 'hCoV-19/Vietnam/040112/2021': 'Others',
 'hCoV-19/Vietnam/050112/2021': 'Others',
 'hCoV-19/Vietnam/060112/2021': 'Others',
 'hCoV-19/Vietnam/080112/2021': 'Others',
 'hCoV-19/Vietnam/090112/2021': 'Others',
 'hCoV-19/Vietnam/110112/2021': 'Others',
 'hCoV-19/Vietnam/140112/2021': 'Others',
 'hCoV-19/Vietnam/030113/2021': 'Others',
 'hCoV-19/Vietnam/070113/2021': 'Others',
 'hCoV-19/Vietnam/080113/2021': 'Others',
 'hCoV-19/Vietnam/110113/2021': 'Others',
 'hCoV-19/Vietnam/160113/2021': 'Others',
 'hCoV-19/Vietnam/170113/2021': 'Others',
 'hCoV-19/Vietnam/190113/2021': 'Others',
 'hCoV-19/Vietnam/200113/2021': 'Others',
 'hCoV-19/Vietnam/010114/2021': 'Others',
 'hCoV-19/Vietnam/050114/2021': 'Others',
 'hCoV-19/Vietnam/060114/2021': 'Others',
 'hCoV-19/Vietnam/070114/2021': 'Others',
 'hCoV-19/V

### Processing Pango lineage file

In [6]:
pango_lineage_results_1 = pd.read_csv('raw_FASTA/Vietnam_results.tsv', sep='\t')


pango_lineage_results = pd.concat([pango_lineage_results_1])
pango_lineage_results = pango_lineage_results.set_index('strain')
pango_lineage_results


a = []
b = []
index1 = []
lineage_2 = []
lineage_1 = []
month = []

for seq_name in pango_lineage_results.index:
    index1.append(seq_name)

for lineage in pango_lineage_results['pangolin_lineage']:
    if lineage.count('.') >= 2:
        a = lineage.split('.')
        b = ".".join(a[0:2])
        lineage_2.append(b)
    else:
        lineage_2.append(lineage)

for lineage in pango_lineage_results['pangolin_lineage']:
    a = lineage.split('.')[0]
    lineage_1.append(a)

    
c = {'Sequence name':index1, 'lineage_1':lineage_1, 'lineage_2':lineage_2}
d = pd.DataFrame(data=c)
d = d.set_index('Sequence name')

pango_lineage_results = pd.concat([pango_lineage_results,d], axis=1)
pango_lineage_results = pango_lineage_results.loc[:,['pangolin_lineage', 'lineage_1', 'lineage_2', 'date_submitted']]
pango_lineage_results




,pangolin_lineage,lineage_1,lineage_2,date_submitted
hCoV-19/Vietnam/HCMC-05049-02/2021,A.23.1,A,A.23,2021-03-26
hCoV-19/Vietnam/39730/2020,B.1.1,B,B.1,2020-11-27
hCoV-19/Vietnam/HCMC-05047-02/2021,A.23.1,A,A.23,2021-03-19
hCoV-19/Vietnam/HCMC-05109070-05/2021,AY.57,AY,AY.57,2021-06-01
hCoV-19/Vietnam/040112/2021,AY.57,AY,AY.57,2021-11-05
...,...,...,...,...
hCoV-19/Vietnam/BD2021_111/2021,AY.57,AY,AY.57,2022-02-08
hCoV-19/Vietnam/BD2021_112/2021,AY.57,AY,AY.57,2022-02-08
hCoV-19/Vietnam/BD2021_113/2021,AY.57,AY,AY.57,2022-02-08
hCoV-19/Vietnam/QB2021_179/2021,AY.57,AY,AY.57,2022-02-09


### Processing decision_tree results

In [7]:
d = {'strain':decision_tree().keys(), 'multiplex qPCR':decision_tree().values()}
decision_tree_dataframe = pd.DataFrame(data=d)
decision_tree_dataframe = decision_tree_dataframe.set_index('strain')
decision_tree_dataframe

,multiplex qPCR
strain,
hCoV-19/Vietnam/HCMC-05049-02/2021,Others
hCoV-19/Vietnam/39730/2020,Others
hCoV-19/Vietnam/HCMC-05047-02/2021,Others
hCoV-19/Vietnam/HCMC-05109070-05/2021,Others
hCoV-19/Vietnam/040112/2021,Others
...,...
hCoV-19/Vietnam/BD2021_110/2021,Others
hCoV-19/Vietnam/BD2021_111/2021,Others
hCoV-19/Vietnam/BD2021_112/2021,Others


### Merge two data frames

In [8]:
merged_results = pd.concat([pango_lineage_results,decision_tree_dataframe], axis=1)
merged_results = merged_results[~merged_results['pangolin_lineage'].isin(['None'])]
merged_results = merged_results.dropna(subset=['pangolin_lineage'])
d = merged_results
d

,pangolin_lineage,lineage_1,lineage_2,date_submitted,multiplex qPCR
hCoV-19/Vietnam/HCMC-05049-02/2021,A.23.1,A,A.23,2021-03-26,Others
hCoV-19/Vietnam/39730/2020,B.1.1,B,B.1,2020-11-27,Others
hCoV-19/Vietnam/HCMC-05047-02/2021,A.23.1,A,A.23,2021-03-19,Others
hCoV-19/Vietnam/HCMC-05109070-05/2021,AY.57,AY,AY.57,2021-06-01,Others
hCoV-19/Vietnam/040112/2021,AY.57,AY,AY.57,2021-11-05,Others
...,...,...,...,...,...
hCoV-19/Vietnam/BD2021_111/2021,AY.57,AY,AY.57,2022-02-08,Others
hCoV-19/Vietnam/BD2021_112/2021,AY.57,AY,AY.57,2022-02-08,Others
hCoV-19/Vietnam/BD2021_113/2021,AY.57,AY,AY.57,2022-02-08,Others
hCoV-19/Vietnam/QB2021_179/2021,AY.57,AY,AY.57,2022-02-09,Others


### Just to check how many Omicron variants we have (by complete sequence)

In [9]:


condition1 = (merged_results['pangolin_lineage'] == 'B.1.1.529')
condition2 = (merged_results['lineage_1'] == 'BA')
omicron = merged_results.loc[condition1 | condition2]


print("omicron", len(omicron))

omicron 120


### Variant assignment by qPCR

In [10]:
decision_tree()
count_decision_tree = {}

def variant_count():
    for variant in decision_tree().values():
        if variant in count_decision_tree:
            count_decision_tree[variant] += 1
        else:
            count_decision_tree[variant] = 1

    print(count_decision_tree)

variant_count()

{'Others': 1887, 'Omicron': 120}


### Sensitivity and Specificity

In [11]:
#Omicron_variants
condition1 = (merged_results['pangolin_lineage'] == 'B.1.1.529')
condition2 = (merged_results['lineage_1'] == 'BA')
a = merged_results.loc[condition1 | condition2]
Omicron_TP = a.loc[a['multiplex qPCR'] == 'Omicron'].count().at['multiplex qPCR']
Omicron_FN = a.loc[a['multiplex qPCR'] != 'Omicron'].count().at['multiplex qPCR']

a = merged_results.loc[~condition1 & ~condition2]
Omicron_FP = a.loc[a['multiplex qPCR'] == 'Omicron'].count().at['multiplex qPCR']
Omicron_TN = a.loc[a['multiplex qPCR'] != 'Omicron'].count().at['multiplex qPCR']

Omicron_sensitivity = Omicron_TP / (Omicron_TP + Omicron_FN)
Omicron_specificity = Omicron_TN / (Omicron_TN + Omicron_FP)

print("Omicron variants sensitivity and specificity", Omicron_sensitivity, Omicron_specificity)
print("Total number", Omicron_TP + Omicron_TN + Omicron_FP + Omicron_FN)
print("TP", Omicron_TP)
print("FN", Omicron_FN)
print("FP", Omicron_FP)
print("TN", Omicron_TN)

Omicron variants sensitivity and specificity 1.0 1.0
Total number 2007
TP 120
FN 0
FP 0
TN 1887


In [12]:
#Total SARS-CoV-2


def Total():
    variant_classification = {}
    variant_qPCR_results = do_primers_work()
    for seq_key in variant_qPCR_results.keys():
        if variant_qPCR_results[seq_key] == [True, True]:
            variant_classification[seq_key] = 'COVID'
        elif variant_qPCR_results[seq_key] == [True, False]:
            variant_classification[seq_key] = 'COVID'
        else:    
            variant_classification[seq_key] = 'Fail'            
    return(variant_classification)

count_decision_tree = {}

def variant_count():
    for variant in Total().values():
        if variant in count_decision_tree:
            count_decision_tree[variant] += 1
        else:
            count_decision_tree[variant] = 1

    print(count_decision_tree)

variant_count()


{'COVID': 1981, 'Fail': 26}
